In [ ]:
# This allows importing Jupyter notebooks as modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import JupyterNotebookImporter

In [ ]:
import numpy as np
import AlgoPlotting as plt

In [ ]:
def shape_it(X):
    return np.expand_dims(X.reshape((-1,1)),2)

In [ ]:
def generate_prediction_data(price_data, sell_target_diff=1.00, stop_loss_diff=1.00, commission=0):
    predictions = []
    N = len(price_data)
    for ii in range(N):
        sell_target = price_data[ii] + sell_target_diff
        stop_loss = price_data[ii] - stop_loss_diff
        jj = ii
        while price_data[jj] >= stop_loss and price_data[jj] <= sell_target and jj < N-1:
            if jj < N-1:
                jj += 1
        if price_data[jj] >= sell_target:
            prediction = 1
        else: # price_data[jj] <= stop_loss:
            prediction = 0
#         elif jj >= N and price_data[N-1] > price_data[ii]:
#             prediction = price_data[N-1]/sell_target
#         else:
#             prediction = -stop_loss/price_data[N-1]
        
        predictions.append(prediction)
    return np.array(predictions)
        

In [ ]:
def generate_data():
    N = 1000
    period = 200 + np.random.rand()*200
    period2 = 50 + np.random.rand()*50
    amp = np.random.rand()
    phase = np.random.rand() * 2 * np.pi
    phase2 = np.random.rand() * 2 * np.pi
    
    x = []
    for ii in range(N):
        bias = amp * np.sin(2 * np.pi*ii/period + phase) + (1 - amp) * np.sin(2 * np.pi*ii/period2 + phase2)
        delta = 3 * 2 * (np.random.rand()-0.5) + bias
        x.append(delta)
    x = np.cumsum(np.array(x))
    
#     N = 1000
#     p = np.cumsum(np.random.rand(N)-0.5)
#     p = (p - np.min(p))/(np.max(p) - np.min(p))
    
    
    x -= min(x)
    x = np.array(x)
    y = generate_prediction_data(x, sell_target_diff=15, stop_loss_diff=15)

    
    
    #x = np.vstack((x0, x1)).transpose()
    #y = np.array(y[10:])
    y = y[1:]
#    x = np.vstack((np.diff(x), np.array(p[:-1]))).transpose()

#    plt.XYChart(y=x, names=['x'])
    x = np.diff(x)
#    plt.XYChart(y=x, names=['x'])
#    plt.XYChart(y=y, names=['y'])
    
#    xy_chart = plt.XYChart(y=[x ,y], names=['x', 'y'])
    y_train = shape_it(y)
#    x_train = x.reshape(x.shape[0], x.shape[1], 1)
    x_train = shape_it(x)
    
#     print(x_train.shape)
#     print(y_train.shape)
    
    return x_train, y_train

In [ ]:
generate_data()

In [ ]:
# TODO
# Learn how to make actual TODO's
# Implement dynamic sell targets and stop losses
# Would be nice to have a cursor callback that plotted the stop loss and sell target at a given point when moused over but sounds like a PITA

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN, LSTM
from keras import optimizers
from keras import regularizers
import random
import numpy as np
import pandas as pd
from operator import add

x_temp, y_temp = generate_data()
batch_input_shape = (1,x_temp.shape[1],1)

In [ ]:
# model = Sequential()
# los = 'binary_crossentropy' # 'mse'
# act = 'tanh'
# learning_rate = 0.00005
# decay_rate = learning_rate/1000
# momentum = 0.0
# sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# opt = sgd
# kreg = regularizers.l2(0.0005)
# areg = regularizers.l2(0.0005)
# neurons = 20
# num_recurrent_layers = 3
# for _ in range(max(num_recurrent_layers - 1, 0)):
#     model.add(SimpleRNN(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True,
#                              kernel_regularizer=kreg, activity_regularizer=areg))
# model.add(SimpleRNN(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True,
#                          kernel_regularizer=kreg, activity_regularizer=areg))
# model.add(Dense(neurons, activation=act, 
#                      kernel_regularizer=kreg, activity_regularizer=areg))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss=los, optimizer=opt)

In [ ]:
model = Sequential()
los = 'binary_crossentropy' # 'mse'
act = 'tanh'
learning_rate = 0.0005
decay_rate = learning_rate/1000
momentum = 0.0
sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = sgd
kreg = regularizers.l2(0.0005)
areg = regularizers.l2(0.0005)
neurons = 20
num_recurrent_layers = 3
for _ in range(max(num_recurrent_layers - 1, 0)):
    model.add(LSTM(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True,
                   kernel_regularizer=kreg, activity_regularizer=areg,
                   unit_forget_bias=True, bias_initializer='zeros'))
model.add(LSTM(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True,
               kernel_regularizer=kreg, activity_regularizer=areg,
               unit_forget_bias=True, bias_initializer='zeros'))
model.add(Dense(neurons, activation=act, 
                kernel_regularizer=kreg, activity_regularizer=areg))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=los, optimizer=opt)

In [ ]:
# Initialize charts
input_chart = plt.XYChart(title='Input', x_label='Index', y_label='X', names=['Test Input ' + str(ii) for ii in range(x_temp.shape[1])])
predictions_chart = plt.XYChart(title='Predictions', x_label='Index', y_label='Y', names=['Prediction', 'Test Data'])
losses_chart = plt.XYChart(title='Losses', x_label='Epochs', y_label='Loss')

In [ ]:
epochs = 5000
losses = []
batch_size = 1
epoch_update_interval = 10
for epoch in range(epochs):
    x_train, y_train = generate_data()
    x_test, y_test = generate_data()
    
    history = model.fit(x_train, np.reshape(y_train,(-1,)), epochs=1,
                              batch_size=1, verbose=0, shuffle=False)
    loss = history.history['loss'][0]
    losses.append(loss)
    
   # Update the plots
    if not epoch % epoch_update_interval:
        model.reset_states()
        predictions = []
        predictions = model.predict(x_test, batch_size=batch_size, verbose=0)
        expected = y_test

        # Push off the starting indices once the losses is long enough
        input_chart.update(y=[x_test[:,ii] for ii in range(x_test.shape[1])])
        predictions_chart.update(y=[predictions, y_test])
        losses_chart.update(y=losses)
    model.reset_states()
